# Import Libraries

In [1]:
import pandas as pd

# Obtain Data

Read csv file with consumer reports ratings into pandas df

In [2]:
cr_df = pd.read_csv('Data/Ratings.csv')
cr_df = cr_df.iloc[:, range(1, len(cr_df.columns), 2)]
cr_df.columns = [x.replace('.1', '') for x in cr_df.columns]
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('mercedes-benz', 'mercedes benz'))
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('mercedes-amg', 'mercedes amg'))
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('land-rover', 'land rover'))
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('alfa-romeo', 'alfa romeo'))
cr_df['Brand'] = cr_df['Model'].apply(lambda x : x.split('-')[0])
def model_name(row):
    return row['Model'].replace(row['Brand']+'-', '')
cr_df['Model'] = cr_df.apply(model_name, axis=1)
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('-', ' '))
print('cr_df shape: ', cr_df.shape)
cr_df.head()

cr_df shape:  (295, 18)


,Model,Reliability,Acceleration,Braking,Ride,Noise,Front Seat Comfort,Rear Seat Comfort,Interior Fit and Finish,Trunk/Cargo Area,Fuel Economy,Routine Handling,Headlights,Driving Position,Third Seat Comfort,Usability,Max. Load,Brand
0,v60,1 / 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,volvo
1,500l,NaN,3 / 5,4 / 5,2 / 5,3 / 5,3 / 5,4 / 5,3 / 5,2 / 5,4 / 5,4 / 5,3 / 5,3 / 5,NaN,3 / 5,860 lb.,fiat
2,xf,NaN,5 / 5,5 / 5,5 / 5,4 / 5,5 / 5,4 / 5,5 / 5,2 / 5,3 / 5,5 / 5,3 / 5,4 / 5,NaN,3 / 5,960 lb.,jaguar
3,tt,NaN,5 / 5,5 / 5,3 / 5,3 / 5,5 / 5,1 / 5,5 / 5,2 / 5,4 / 5,5 / 5,2 / 5,4 / 5,NaN,2 / 5,770 lb.,audi
4,mirai,NaN,3 / 5,3 / 5,4 / 5,4 / 5,4 / 5,3 / 5,4 / 5,2 / 5,5 / 5,2 / 5,4 / 5,3 / 5,NaN,3 / 5,690 lb.,toyota


Read csv file with auto sales data into pandas df

In [3]:
def read_sales_excel(file, column, header):
    sales_df = pd.read_excel(file, header=header)
    sales_df.rename(columns={'Unnamed: 0': 'Model'}, inplace=True)
    sales_df.dropna(subset=[column], inplace=True)
    sales_df = sales_df[['Model', column]]
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('(D)', ''))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("(I)", ""))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("*", ""))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("‡", ""))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("†", ""))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("Total", ""))
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.strip())
    sales_df['Model'] = sales_df['Model'].apply(lambda x: x.lower())
    sales_df = sales_df.groupby('Model').agg({'Model':'first', column:'max'})
    sales_df.reset_index(inplace=True, drop=True)
    return sales_df
    
    
sales_df = read_sales_excel('Data/Auto_Sales/12-19-auto-sales.xls', '2019.1', 5)
sales_df.rename(columns={'2019.1': 'Sales'}, inplace=True)
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('-', ' '))
sales_df.head()

,Model,Sales
0,2 series,8015.0
1,200,48.0
2,3 series,47827.0
3,300,29213.0
4,370z,2384.0


Reformat sales_df model names to match cr_df model names for mergeer

In [4]:
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('mazda3', '3'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('mazda6', '6'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('f series', 'f 150'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('silverado', 'silverado 1500'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('ram pickup', '1500'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('discovery/lr4', 'discovery'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('spider', '124'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('e class/cls', 'e class'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('86 (incl. fr s)', '86'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('nautilus/mkx', 'nautilus'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('a4/s4', 'a4'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('a5/s5', 'a5'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('a6/s6', 'a6'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('a7/s7', 'a7'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('a8/s8', 'a8'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('718', '718 boxster'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('countryman', 'cooper countryman'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('cooper s', 'cooper'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('90 series', 's90'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('60 series', 's60'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('370z', 'z'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('sierra', 'sierra 1500'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('slc/slk', 'slc'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('rl/rlx', 'rlx'))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('glc/glk', 'glc'))
sales_df['Model'] = sales_df['Model'].replace({'gt': 'amg gt'})
sales_df = sales_df[sales_df['Model']!='golf']
sales_df.reset_index(drop=True, inplace=True)
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace('golf/golf r/gti', 'golf'))
sales_df.head()

,Model,Sales
0,2 series,8015.0
1,200,48.0
2,3 series,47827.0
3,300,29213.0
4,z,2384.0


Merge sales_df and cr_df files

In [5]:
merged_df = pd.merge(sales_df, cr_df, on='Model')
merged_df.head()

,Model,Sales,Reliability,Acceleration,Braking,Ride,Noise,Front Seat Comfort,Rear Seat Comfort,Interior Fit and Finish,Trunk/Cargo Area,Fuel Economy,Routine Handling,Headlights,Driving Position,Third Seat Comfort,Usability,Max. Load,Brand
0,2 series,8015.0,NaN,5 / 5,5 / 5,3 / 5,4 / 5,5 / 5,2 / 5,4 / 5,2 / 5,4 / 5,5 / 5,1 / 5,4 / 5,NaN,4 / 5,805 lb.,bmw
1,3 series,47827.0,5 / 5,5 / 5,5 / 5,3 / 5,4 / 5,5 / 5,3 / 5,5 / 5,3 / 5,4 / 5,5 / 5,2 / 5,4 / 5,NaN,3 / 5,825 lb.,bmw
2,300,29213.0,5 / 5,4 / 5,4 / 5,4 / 5,5 / 5,5 / 5,5 / 5,5 / 5,3 / 5,3 / 5,4 / 5,1 / 5,4 / 5,NaN,4 / 5,865 lb.,chrysler
3,z,2384.0,NaN,5 / 5,5 / 5,2 / 5,2 / 5,4 / 5,NaN,5 / 5,1 / 5,3 / 5,5 / 5,4 / 5,2 / 5,NaN,4 / 5,450 lb.,nissan
4,4 series,18621.0,5 / 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,790 lb.,bmw
